In [2]:
import numpy as np
from scipy.optimize import fsolve
import time


In [7]:
def binary_search_rate(target_value, initial_balance, monthly_investment, months, tolerance=1e-10):
    left, right = -0.99, 10.0
    
    def calc_final_value(r):
        if abs(r) < 1e-10:
            return initial_balance + monthly_investment * months
        return (initial_balance * pow(1 + r, months) + 
                monthly_investment * (pow(1 + r, months) - 1) / r)
    
    while right - left > tolerance:
        mid = (left + right) / 2
        final_value = calc_final_value(mid)
        
        if (final_value - target_value < tolerance) and final_value >= target_value:
            return mid
        elif final_value < target_value:
            left = mid
        else:
            right = mid
    monthly_ = (left + right) / 2
    annual = pow(1 + monthly_, 12) - 1

    return annual



In [8]:
def broyden_method(target_value, initial_balance, monthly_investment, months):
    def objective(r):
        if abs(r) < 1e-10:
            return initial_balance + monthly_investment * months - target_value
        return (initial_balance * pow(1 + r, months) + 
                monthly_investment * (pow(1 + r, months) - 1) / r) - target_value
    
    return fsolve(objective, x0=0.01)[0]

In [9]:
# 测试参数
test_cases = [
    (1000, 0, 10, 12),  # 10年
    (5000000, 0, 2000, 240),  # 20年
    (10000000, 0, 5000, 360)  # 30年
]

In [10]:
# 性能比较
print("Performance Comparison:")
print("-" * 50)
print("Case | Binary Search | Broyden Method")
print("-" * 50)

for target, initial, monthly, months in test_cases:
    # 二分查找
    start_time = time.time()
    binary_result = binary_search_rate(target, initial, monthly, months)
    binary_time = time.time() - start_time
    
    # Broyden方法
    start_time = time.time()
    broyden_result = broyden_method(target, initial, monthly, months)
    broyden_time = time.time() - start_time
    
    print(f"{target:,} | {binary_time:.6f}s | {broyden_time:.6f}s")
    print(f"Results: {binary_result:.6f} vs {broyden_result:.6f}")
    print("-" * 50)

Performance Comparison:
--------------------------------------------------
Case | Binary Search | Broyden Method
--------------------------------------------------
1,000 | 0.000052s | 0.001163s
Results: 34.709691 vs 0.347097
--------------------------------------------------
5,000,000 | 0.000030s | 0.000204s
Results: 0.201979 vs 0.015449
--------------------------------------------------
10,000,000 | 0.000027s | 0.000165s
Results: 0.098400 vs 0.007852
--------------------------------------------------
